## Ensemble Bagging Classifier

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [3]:
x,y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

In [4]:
x_train, x_test, y_train,  y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [5]:
x_train.shape, x_test.shape

((800, 20), (200, 20))

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB


In [7]:
lr = LogisticRegression()
svc= SVC()
dtc = DecisionTreeClassifier()
nb = GaussianNB()

In [8]:
from sklearn.ensemble import VotingClassifier
#  parameters -> estimators - list of models, voting - (hard: prediction class labels, soft: prediction probability)

In [9]:
ensemble_clf = VotingClassifier(estimators=[('log_reg',lr), ('support vect clf',svc), ('decision tree', dtc), ('naive bayes', nb)])

In [10]:
ensemble_clf 
# family of algorithm   

VotingClassifier(estimators=[('log_reg', LogisticRegression()),
                             ('support vect clf', SVC()),
                             ('decision tree', DecisionTreeClassifier()),
                             ('naive bayes', GaussianNB())])

In [11]:
ensemble_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('log_reg', LogisticRegression()),
                             ('support vect clf', SVC()),
                             ('decision tree', DecisionTreeClassifier()),
                             ('naive bayes', GaussianNB())])

In [12]:
y_pred = ensemble_clf.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.845

## Ensemble custom Bagging Regressor

In [14]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

x,y = make_regression(n_samples=1000, n_features=10, noise = 0.1, random_state=42)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=20)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor

In [17]:
lr = LinearRegression()
dtr = DecisionTreeRegressor()
svr = SVR(kernel='linear')
model = VotingRegressor(estimators=[('LinearRegression', lr),('DecisionTreeRegressor', dtr),('SuppotVectRegression', svr)])

In [18]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
# below confusion matrix is for classification only
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred)

## Model training, pipeline & tranformation

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [20]:
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [21]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [22]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
df['time'] = enc.fit_transform(df['time'])
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [23]:
df.time.unique()#dinner>0, lunch>1

array([0, 1])

In [24]:
x = df.drop('time', axis =1)
y = df['time']

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

In [26]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [27]:
# handling missing values - IMPUTATION
from sklearn.impute import SimpleImputer

# data encoding
from sklearn.preprocessing import OneHotEncoder

# features scaling
from sklearn.preprocessing import StandardScaler

#  pipeline >>> a sequence of data transformations
# column transformer>>> groups all the pipeline steps for each of the columns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [28]:
# Different encoding transformations is used for categorical columns and numerical columns.
# Hence, we need to segregate the columns.
categorical_columns = ['sex', 'smoker', 'day']
numerical_columns = ['total_bill', 'tip', 'size']

In [29]:
#feature engineering automation using pipeline and column transformer.

# how to make a pipeline

# pipeline for numeric data columns
num_pipeline = Pipeline(steps=[('imputation',SimpleImputer(strategy='median')),
                               ('scaling',StandardScaler())])
cate_pipeline = Pipeline(steps=[('imputation',SimpleImputer(strategy='most_frequent')),
                                 ('encoding',OneHotEncoder())])

In [30]:
preprocessor = ColumnTransformer([('num_pipelines',num_pipeline,numerical_columns),
                   ('cate_pipelines',cate_pipeline,categorical_columns)])

In [31]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


models = {"SVC": SVC(),
         "dtc": DecisionTreeClassifier(),
         "Logreg": LogisticRegression()}

In [33]:
from sklearn.metrics import accuracy_score

def model_train_eval(x_train, y_train,x_test, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        evaluation[list(models.keys())[i]] = accuracy_score(y_test, y_pred)
    return evaluation
model_train_eval(x_train, y_train,x_test, y_test, models)

{'SVC': 0.9183673469387755,
 'dtc': 0.8979591836734694,
 'Logreg': 0.9183673469387755}

## Bagging Ensemble

### Random Forest Classifier

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
rf = RandomForestClassifier()

In [36]:
rf.fit(x_train, y_train)

RandomForestClassifier()

In [37]:
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.8979591836734694

In [38]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth' : [1, 2, 3,4, 10, 20, 30],
         'n_estimators': [50, 100, 200, 300, 400, 1000],
         'criterion': ["gini", 'entropy']}

In [39]:
gscv = GridSearchCV(rf, param_grid=params, cv = 5, scoring='accuracy', verbose=3)

In [40]:
gscv.fit(x_train, y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV 1/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.769 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.974 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.795 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=1, n_estimators=50;, score=1.000 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.923 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.974 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.949 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.846 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 10, 20, 30],
                         'n_estimators': [50, 100, 200, 300, 400, 1000]},
             scoring='accuracy', verbose=3)

In [41]:
gscv.best_score_

np.float64(0.9794871794871796)

In [42]:
gscv.best_params_

{'criterion': 'gini', 'max_depth': 1, 'n_estimators': 200}

### OOB Score (out of bag Score)

In [43]:
from sklearn.datasets import make_classification

In [44]:
x,y = make_classification(n_samples=1000, n_features=30, n_classes=2, random_state=1)

In [45]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True,random_state=1)

In [46]:
rf.fit(x_train, y_train)

RandomForestClassifier(oob_score=True, random_state=1)

In [47]:
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.8979591836734694

In [48]:
rf.oob_score_

0.9641025641025641

## Random Forest Regressor
use total_bill as target - regressor problem


In [ ]:
data = sns.load_dataset('tips')
data.head()
preprocessor = ColumnTransformer([('num_pipelines',num_pipeline,numerical_columns),
                   ('cate_pipelines',cate_pipeline,categorical_columns)])


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [55]:
x,y = data.drop(columns=['total_bill']), data['total_bill']
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [56]:
x_train

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
154,2.00,Male,No,Sun,Dinner,4
167,4.50,Male,No,Sun,Dinner,4
110,3.00,Male,No,Sat,Dinner,2
225,2.50,Female,Yes,Fri,Lunch,2
...,...,...,...,...,...,...
137,2.00,Female,No,Thur,Lunch,2
72,3.14,Female,Yes,Sat,Dinner,2
140,3.50,Female,No,Thur,Lunch,2
235,1.25,Male,No,Sat,Dinner,2


In [60]:
preprocessor = ColumnTransformer([('cate_pipelines',cate_pipeline,categorical_columns)])
x_train= preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [61]:
x_train

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.]])

In [62]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)

RandomForestRegressor()

In [63]:
y_pred = rfr.predict(x_test)
